In [32]:
#pip install urllib3 pip install requests pip install time pip install http.cookiejar pip install urllib pip install getpass

In [33]:
#Import all necessary libraries

import sys
import json
import urllib3
import certifi
import requests
from time import sleep
from http.cookiejar import CookieJar
import urllib.request
from urllib.parse import urlencode
import getpass

#For authentification
import os
import earthaccess

In [34]:


os.environ['EARTHDATA_USERNAME'] = 'tnourlyn'
os.environ['EARTHDATA_PASSWORD'] = 'Lutinette2411%'
try:
    auth = earthaccess.login(strategy="environment")
    print("✅ Authentification réussie !")
except Exception as e:
    print(f"❌ Échec de login : {e}")

✅ Authentification réussie !


In [35]:
# Create a urllib PoolManager instance to make requests.
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
# Set the URL for the GES DISC subset service endpoint
url = 'https://disc.gsfc.nasa.gov/service/subset/jsonwsp' #access the subsetting service of earthdata

In [41]:
#JSON formatted web protocol request sent to GES DISC
def get_http_data(request):
    hdrs = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    data = json.dumps(request)
    r = http.request('POST', url, body=data, headers=hdrs)
    response = json.loads(r.data)

    # Check for API fault
    if response.get('type') == 'jsonwsp/fault':
        print('❌ API Error')
        print(json.dumps(response, indent=2))
        sys.exit(1)

    return response

In [50]:
# Define the parameters for the data subset
product = 'NASA/FLDAS/NOAH01/C/GL/M/V001'
#includes short names for all variables except for longitude, latitude and time which are used to index the data rather than give info 
varNames =[
    'Evap_tavg',
    'LWdown_f_tavg',
    'Lwnet_tavg',
    'Psurf_f_tavg',
    'Qair_f_tavg',
    'Qg_tavg',
    'Qh_tavg',
    'Qle_tavg',
    'Qs_tavg',
    'Qsb_tavg',
    'RadT_tavg',
    'Rainf_f_tavg',
    'SWE_inst',
    'SWdown_f_tavg',
    'SnowCover_inst',
    'SnowDepth_inst',
    'Snowf_tavg',
    'Swnet_tavg',
    'Tair_f_tavg',
    'Wind_f_tavg',
    'SoilMoi00_10cm_tavg',
    'SoilMoi10_40cm_tavg',
    'SoilMoi40_100cm_tavg',
    'SoilMoi100_200cm_tavg',
    'SoilTemp00_10cm_tavg',
    'SoilTemp10_40cm_tavg',
    'SoilTemp40_100cm_tavg',
    'SoilTemp100_200cm_tavg'
]
minlon = 11.7 #Bouding box info for Nigeria
maxlon = 12.1
minlat = 13.0
maxlat = 13.4
begTime = '1982-01-01'
endTime = '2026-01-01'


In [51]:
# Construct JSON WSP request for API method: subset
subset_request = {
    'methodname': 'subset',
    'type': 'jsonwsp/request',
    'version': '1.0',
    'args': {
        'role'  : 'subset',
        'start' : begTime,
        'end'   : endTime,
        'box'   : [minlon, minlat, maxlon, maxlat],
        'crop'  : True,
        'data': [{'datasetId': product, 'variable': var} for var in varNames]
    }
}

In [52]:
# Submit the subset request to the GES DISC Server
response = get_http_data(subset_request)

# Report the JobID and initial status
myJobId = response['result']['jobId']
print('Job ID: '+myJobId)
print('Job status: '+response['result']['Status'])

❌ API Error
{
  "type": "jsonwsp/fault",
  "version": "1.0",
  "fault": {
    "code": "server",
    "string": "Failed to find a service configuration matching requested constraints."
  }
}


SystemExit: 1

We find this method dooes not seem to work to extract the data 

In [49]:
# STEP 7
# Construct JSON WSP request for API method: GetStatus
status_request = {
    'methodname': 'GetStatus',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {'jobId': myJobId}
}

# Check on the job status after a brief nap
while response['result']['Status'] in ['Accepted', 'Running']:
    sleep(5)
    response = get_http_data(status_request)
    status  = response['result']['Status']
    percent = response['result']['PercentCompleted']
    print ('Job status: %s (%d%c complete)' % (status,percent,'%'))
if response['result']['Status'] == 'Succeeded' :
    print ('Job Finished:  %s' % response['result']['message'])
else : 
    print('Job Failed: %s' % response['fault']['code'])
    sys.exit(1)

KeyError: 'Status'

In [ ]:
# STEP 8 (Plan A - preferred)
# Construct JSON WSP request for API method: GetResult
batchsize = 20
results_request = {
    'methodname': 'GetResult',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {
        'jobId': myJobId,
        'count': batchsize,
        'startIndex': 0
    }
}

# Retrieve the results in JSON in multiple batches 
# Initialize variables, then submit the first GetResults request
# Add the results from this batch to the list and increment the count
results = []
count = 0 
response = get_http_data(results_request) 
count = count + response['result']['itemsPerPage']
results.extend(response['result']['items']) 

# Increment the startIndex and keep asking for more results until we have them all
total = response['result']['totalResults']
while count < total :
    results_request['args']['startIndex'] += batchsize 
    response = get_http_data(results_request) 
    count = count + response['result']['itemsPerPage']
    results.extend(response['result']['items'])
       
# Check on the bookkeeping
print('Retrieved %d out of %d expected items' % (len(results), total))

Retrieved 6 out of 6 expected items


In [ ]:
# Sort the results into documents and URLs

docs = []
urls = []
for item in results :
    try:
        if item['start'] and item['end'] : urls.append(item) 
    except:
        docs.append(item)
# Print out the documentation links, but do not download them
# print('\nDocumentation:')
# for item in docs : print(item['label']+': '+item['link'])

In [ ]:
# STEP 10 
# Use the requests library to submit the HTTP_Services URLs and write out the results.
print('\nHTTP_services output:')
for item in urls :
    URL = item['link'] 
    result = requests.get(URL)
    try:
        result.raise_for_status()
        outfn = item['label']
        f = open(outfn,'wb')
        f.write(result.content)
        f.close()
        print(outfn, "is downloaded")
    except:
        print('Error! Status code is %d for this URL:\n%s' % (result.status.code,URL))
        print('Help for downloading data is at https://disc.gsfc.nasa.gov/data-access')
        
print('Downloading is done and find the downloaded files in your current working directory')


HTTP_services output:


AttributeError: 'Response' object has no attribute 'status'